# Create CDAW CME Merge To LASCO Catalog
This notebook is for creating a merged CME/LASCO catalogs of both CME events and time intervals when there are no CMEs (depends on telescope). 

In [1]:
import requests
import pandas as pd

Need_To_Redownload = False

LASCO_Catalog_Filename = '../lasco_img_catalog_lvl_1.txt'
CME_Catalog_Filename = '../cme_event_catalog.csv'
Merged_Catalog_Filename = '../merged_cdaw_lasco_catalog.csv'

# sampling of non-events
C2_non_CME_Catalog_Filename = '../c2_non_events.csv'
C3_non_CME_Catalog_Filename = '../c3_non_events.csv'

In [2]:
# pull the lasco catalog to local disk
if Need_To_Redownload:
    
    # download LASCO catalog now
    lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/level_1/img_hdr.txt'
    response = requests.get(lasco_hdr_url, allow_redirects=True)
    
    # cache locally
    open(LASCO_Catalog_Filename, 'wb').write(response.content)


In [3]:
cme_data = pd.read_csv(CME_Catalog_Filename)
#cme_data = cme_data.rename(columns={'Unnamed: 0':'CDAW_Id'})
cme_data

,CDAW_Id,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,...,Remarks,Duration_C2,Duration_C3,StartTime_C3_Delta,width,linear,mpa,mass,kinetic,accel
0,5,1996/01/31,06:52:13,274,47,158,219,99,0,-12.3*,...,Poor Event; Only 3 points\n,26582.278481,116518.987342,9746.835443,47,158,272,2.500000e+14,3.200000e+28,-12.3
1,8,1996/02/12,05:47:26,91,53,160,100,211,236,2.0*,...,Poor Event\n,26250.000000,115062.500000,9625.000000,53,160,92,1.300000e+15,1.600000e+29,2.0
2,9,1996/02/17,02:06:31,279,73,317,257,378,532,9.9*,...,Very Poor Event\n,13249.211356,58075.709779,4858.044164,73,317,274,3.300000e+14,1.600000e+29,9.9
3,12,1996/03/02,04:11:53,88,36,108,95,120,177,1.0*,...,Poor Event\n,38888.888889,170462.962963,14259.259259,36,108,86,1.500000e+14,8.700000e+27,1.0
4,16,1996/03/07,07:51:56,91,43,60,60,61,75,0.1*,...,Poor Event\n,70000.000000,306833.333333,25666.666667,43,60,87,0.000000e+00,0.000000e+00,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11093,31613,2021/11/23,11:12:10,172,75,199,159,238,376,4.8*,...,Very Poor Event\n,21105.527638,92512.562814,7738.693467,75,199,169,0.000000e+00,0.000000e+00,4.8
11094,31614,2021/11/24,14:26:00,193,211,390,324,458,447,4.0,...,Partial Halo\n,10769.230769,47205.128205,3948.717949,211,390,171,0.000000e+00,0.000000e+00,4.0
11095,31618,2021/11/26,01:25:44,52,63,436,509,362,0,-20.7*,...,Poor Event\n,9633.027523,42224.770642,3532.110092,63,436,67,0.000000e+00,0.000000e+00,-20.7
11096,31619,2021/11/26,05:48:05,137,37,391,487,294,0,-10.4*,...,Poor Event\n,10741.687980,47084.398977,3938.618926,37,391,130,0.000000e+00,0.000000e+00,-10.4


In [4]:
# now work to merge in information from LASCO img catalog
lasco_catalog = pd.read_fwf(LASCO_Catalog_Filename, index=False,
                            names=['filename', 'date-obs', 'time-obs','telescope', 'expTime',
                                   'NumCols', 'NumRows', 'StartCol', 'StartRow', 'Filter', 
                                   'Polarizer','Observ.Seq', 'LEB Program', 'junk'])

In [5]:
lasco_catalog

,filename,date-obs,time-obs,telescope,expTime,NumCols,NumRows,StartCol,StartRow,Filter,Polarizer,Observ.Seq,LEB Program,junk
0,35000706.fts,1996/01/11,22:54:31,C3,3.8,128,128,469,297,Clear,Clear,XZR,0.2298,0
1,35000707.fts,1996/01/11,23:00:29,C3,3.1,512,512,277,275,Clear,Clear,XZR,0.2298,0
2,35000708.fts,1996/01/11,23:15:34,C3,15.1,512,256,277,403,Clear,Clear,XZR,0.2298,0
3,35002640.fts,1996/01/16,00:58:18,C3,19.1,1024,1024,20,1,Clear,Clear,XZR,0.2298,0
4,35002645.fts,1996/01/16,02:10:12,C3,19.1,1024,1024,20,1,Clear,Clear,XZR,0.2298,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913203,35513587.fts,2017/08/31,23:28:54,C3,17.9,1024,1024,20,1,Clear,Clear,XZR,0.7160,0
913204,25649103.fts,2017/08/31,23:34:50,C2,25.1,1024,1024,20,1,Orang,Clear,XZ%,1.4430,0
913205,35513588.fts,2017/08/31,23:40:50,C3,17.6,1024,1024,20,1,Clear,Clear,XZR,0.7150,0
913206,25649104.fts,2017/08/31,23:46:51,C2,25.1,1024,1024,20,1,Orang,Clear,XZ%,1.4400,0


In [6]:
# Drop C1, EIT observations (this speeds up datetime calc below)
lasco_catalog = lasco_catalog[lasco_catalog['telescope'] != 'C1']
lasco_catalog = lasco_catalog[lasco_catalog['telescope'] != 'EIT']

#date_filt = lasco_catalog[lasco_catalog['date-obs']=='2020/05/28']
#date_filt[date_filt['time-obs']=='21:12:11']
len(lasco_catalog)

913208

In [7]:
from datetime import datetime, date, time, timezone, timedelta

# add column of python datetime for easier handling
datetime_col = []
for row in lasco_catalog.iterrows():
    date_obs = row[1]['date-obs']
    time_obs = row[1]['time-obs']
    
    date_obs = [int(t) for t in row[1]['date-obs'].split('/')]
    time_obs = [int(t) for t in row[1]['time-obs'].split(':')]
    
    datetime_col.append(datetime(year=date_obs[0], month=date_obs[1], day=date_obs[2], hour=time_obs[0], minute=time_obs[1], second=time_obs[2]))

lasco_catalog['datetime'] = datetime_col
lasco_catalog.index = lasco_catalog.datetime
lasco_catalog

,filename,date-obs,time-obs,telescope,expTime,NumCols,NumRows,StartCol,StartRow,Filter,Polarizer,Observ.Seq,LEB Program,junk,datetime
datetime,,,,,,,,,,,,,,,
1996-01-11 22:54:31,35000706.fts,1996/01/11,22:54:31,C3,3.8,128,128,469,297,Clear,Clear,XZR,0.2298,0,1996-01-11 22:54:31
1996-01-11 23:00:29,35000707.fts,1996/01/11,23:00:29,C3,3.1,512,512,277,275,Clear,Clear,XZR,0.2298,0,1996-01-11 23:00:29
1996-01-11 23:15:34,35000708.fts,1996/01/11,23:15:34,C3,15.1,512,256,277,403,Clear,Clear,XZR,0.2298,0,1996-01-11 23:15:34
1996-01-16 00:58:18,35002640.fts,1996/01/16,00:58:18,C3,19.1,1024,1024,20,1,Clear,Clear,XZR,0.2298,0,1996-01-16 00:58:18
1996-01-16 02:10:12,35002645.fts,1996/01/16,02:10:12,C3,19.1,1024,1024,20,1,Clear,Clear,XZR,0.2298,0,1996-01-16 02:10:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31 23:28:54,35513587.fts,2017/08/31,23:28:54,C3,17.9,1024,1024,20,1,Clear,Clear,XZR,0.7160,0,2017-08-31 23:28:54
2017-08-31 23:34:50,25649103.fts,2017/08/31,23:34:50,C2,25.1,1024,1024,20,1,Orang,Clear,XZ%,1.4430,0,2017-08-31 23:34:50
2017-08-31 23:40:50,35513588.fts,2017/08/31,23:40:50,C3,17.6,1024,1024,20,1,Clear,Clear,XZR,0.7150,0,2017-08-31 23:40:50


In [8]:
# split up catalog by C2 and C3
c2_lasco = lasco_catalog[lasco_catalog['telescope']=='C2']
c3_lasco = lasco_catalog[lasco_catalog['telescope']=='C3']

In [9]:
# Do the merge of catalogs, using CME events for lookup of filename in LASCO.
# We are not just getting CME images, howerver. For purposes of background subtraction we need to 
# capture the image before first CME event image

cme_ids=[]
filenames=[]
filters = []
dates=[]
times=[]

lasco_keepCols = ['filename', 'telescope', 'NumCols', 'NumRows', 'Filter', 'Polarizer', 'datetime', 'expTime', 'date-obs']
cdaw_keepCols = ['Remarks', 'Central', 'Width', 'Linear', 'CDAW_Id']
bkgnd_keepCols = ['filename', 'NumCols', 'NumRows', 'Filter', 'datetime', 'expTime', 'Polarizer']
bckgnd_fileinfo = [ 'bkgnd_'+colname for colname in bkgnd_keepCols ]

merged_data = {  col:[] for col in lasco_keepCols + cdaw_keepCols + bckgnd_fileinfo }

for event in cme_data.iterrows():
    
    # grab date-time-duration info for our event
    cme_start_date = [int(t) for t in event[1]['Date'].split('/')]
    cme_start_time = [int(t) for t in event[1]['Time'].split(':')]
    cme_c3_delta = timedelta(seconds=event[1]['StartTime_C3_Delta'])
    cme_c2_duration = timedelta(seconds=float(event[1]['Duration_C2']))
    cme_c3_duration = timedelta(seconds=float(event[1]['Duration_C3']))
    
    # convert to Python datetime for easier slicing
    cme_date = date (year=cme_start_date[0], month=cme_start_date[1], day=cme_start_date[2])
    cme_time = time(hour=cme_start_time[0],minute=cme_start_time[1],second=cme_start_time[2])
    
    c2_start = datetime.combine(cme_date, cme_time)
    c2_end   = c2_start + cme_c2_duration
    
    c3_start = c2_start + cme_c3_delta
    c3_end   = c3_start + cme_c3_duration
    
    # get C2 entries
    
    # capture prior image, store with cme image id
    previous_c2_image = c2_lasco[c2_lasco['datetime']<c2_start].iloc[-1]
    
    # and a slice of all of the images which follow
    c2_slice = c2_lasco[c2_lasco['datetime']>c2_start]
    for lasco_row in c2_slice[c2_slice['datetime']<=c2_end].iterrows():
        
        for col in lasco_keepCols:
            merged_data[col].append(lasco_row[1][col])
          
        for col in cdaw_keepCols:
            merged_data[col].append(event[1][col])
        
        #  add in background file to use
        for col in bkgnd_keepCols: 
            merged_data['bkgnd_'+col].append(previous_c2_image[col])
    

    # get C3 entries
    
    # capture prior image, store with cme image id
    previous_c3_image = c3_lasco[c3_lasco['datetime']<c3_start].iloc[-1]
    
    c3_slice = c3_lasco[c3_lasco['datetime']>c3_start]
    for lasco_row in c3_slice[c3_slice['datetime']<=c3_end].iterrows():
        
        for col in lasco_keepCols:
            merged_data[col].append(lasco_row[1][col])
          
        for col in cdaw_keepCols:
            merged_data[col].append(event[1][col])
         
        #  add in background file to use
        for col in bkgnd_keepCols: 
            merged_data['bkgnd_'+col].append(previous_c3_image[col])
    
# convert results to DataFrame
result = pd.DataFrame(merged_data)
result

,filename,telescope,NumCols,NumRows,Filter,Polarizer,datetime,expTime,date-obs,Remarks,...,Width,Linear,CDAW_Id,bkgnd_filename,bkgnd_NumCols,bkgnd_NumRows,bkgnd_Filter,bkgnd_datetime,bkgnd_expTime,bkgnd_Polarizer
0,25000294.fts,C2,1024,1024,Orang,Clear,1996-01-31 08:01:38,25.4,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
1,25000295.fts,C2,1024,1024,Orang,Clear,1996-01-31 09:11:14,25.0,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
2,25000296.fts,C2,1024,1024,Orang,Clear,1996-01-31 10:20:39,25.0,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
3,25000297.fts,C2,1024,1024,Orang,Clear,1996-01-31 11:30:19,25.9,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
4,25000298.fts,C2,1024,1024,Orang,Clear,1996-01-31 12:40:17,25.1,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593204,35513585.fts,C3,1024,1024,Clear,Clear,2017-08-31 23:04:45,17.6,2017/08/31,Poor Event\n,...,54,274,28858,35513550.fts,1024,1024,Clear,2017-08-31 16:17:54,17.6,Clear
593205,35513586.fts,C3,1024,1024,Clear,Clear,2017-08-31 23:16:36,17.5,2017/08/31,Poor Event\n,...,54,274,28858,35513550.fts,1024,1024,Clear,2017-08-31 16:17:54,17.6,Clear
593206,35513587.fts,C3,1024,1024,Clear,Clear,2017-08-31 23:28:54,17.9,2017/08/31,Poor Event\n,...,54,274,28858,35513550.fts,1024,1024,Clear,2017-08-31 16:17:54,17.6,Clear
593207,35513588.fts,C3,1024,1024,Clear,Clear,2017-08-31 23:40:50,17.6,2017/08/31,Poor Event\n,...,54,274,28858,35513550.fts,1024,1024,Clear,2017-08-31 16:17:54,17.6,Clear


In [10]:
# save the merged catalog
result.to_csv(Merged_Catalog_Filename)

In [11]:
# test it out, try to pull a few intervals for events
C2_data = result[result['telescope']=='C2']
C2_data[:10]

,filename,telescope,NumCols,NumRows,Filter,Polarizer,datetime,expTime,date-obs,Remarks,...,Width,Linear,CDAW_Id,bkgnd_filename,bkgnd_NumCols,bkgnd_NumRows,bkgnd_Filter,bkgnd_datetime,bkgnd_expTime,bkgnd_Polarizer
0,25000294.fts,C2,1024,1024,Orang,Clear,1996-01-31 08:01:38,25.4,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
1,25000295.fts,C2,1024,1024,Orang,Clear,1996-01-31 09:11:14,25.0,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
2,25000296.fts,C2,1024,1024,Orang,Clear,1996-01-31 10:20:39,25.0,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
3,25000297.fts,C2,1024,1024,Orang,Clear,1996-01-31 11:30:19,25.9,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
4,25000298.fts,C2,1024,1024,Orang,Clear,1996-01-31 12:40:17,25.1,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
5,25000299.fts,C2,1024,1024,Orang,Clear,1996-01-31 13:50:01,25.1,1996/01/31,Poor Event; Only 3 points\n,...,47,158,5,25000292.fts,1024,1024,Orang,1996-01-31 05:42:43,25.1,Clear
25,25000487.fts,C2,1024,1024,Orang,Clear,1996-02-12 07:01:57,25.1,1996/02/12,Poor Event\n,...,53,160,8,25000485.fts,1024,1024,Orang,1996-02-12 04:32:50,25.1,Clear
26,25000488.fts,C2,1024,1024,Orang,Clear,1996-02-12 08:16:27,25.0,1996/02/12,Poor Event\n,...,53,160,8,25000485.fts,1024,1024,Orang,1996-02-12 04:32:50,25.1,Clear
27,25000489.fts,C2,1024,1024,Orang,Clear,1996-02-12 09:30:48,25.1,1996/02/12,Poor Event\n,...,53,160,8,25000485.fts,1024,1024,Orang,1996-02-12 04:32:50,25.1,Clear
28,25000490.fts,C2,1024,1024,Orang,Clear,1996-02-12 10:45:46,25.1,1996/02/12,Poor Event\n,...,53,160,8,25000485.fts,1024,1024,Orang,1996-02-12 04:32:50,25.1,Clear


In [12]:
#c2_lasco

# now build out the catalog of NON-CME times

In [13]:
# now build out the catalog of NON-CME times
keepCols = ['filename', 'datetime', 'date-obs', 'NumCols', 'NumRows', 'Filter', 'Polarizer', 'expTime', 'telescope']
c2_intervals = c2_lasco[keepCols]
c2_intervals.set_index('datetime')

c3_intervals = c3_lasco[keepCols]
c3_intervals.set_index('datetime')

c2_intervals

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime,telescope
datetime,,,,,,,,,
1996-01-23 17:47:06,25000161.fts,1996-01-23 17:47:06,1996/01/23,512,512,Orang,Clear,74.0,C2
1996-01-23 20:01:08,25000174.fts,1996-01-23 20:01:08,1996/01/23,512,512,Orang,Clear,3.8,C2
1996-01-23 20:06:58,25000175.fts,1996-01-23 20:06:58,1996/01/23,512,512,Orang,Clear,3.8,C2
1996-01-23 21:36:17,25000185.fts,1996-01-23 21:36:17,1996/01/23,512,512,Orang,Clear,3.1,C2
1996-01-24 15:43:29,25000186.fts,1996-01-24 15:43:29,1996/01/24,1024,1024,Orang,Clear,17.1,C2
...,...,...,...,...,...,...,...,...,...
2017-08-31 22:34:39,25649100.fts,2017-08-31 22:34:39,2017/08/31,1024,1024,Orang,Clear,25.1,C2
2017-08-31 23:10:42,25649101.fts,2017-08-31 23:10:42,2017/08/31,1024,1024,Orang,Clear,25.1,C2
2017-08-31 23:22:37,25649102.fts,2017-08-31 23:22:37,2017/08/31,1024,1024,Orang,Clear,25.5,C2


In [14]:
# step through CDAW catalog and calculate when there is NOT an event
    
c2_non_events = { k:[] for k in c2_intervals.columns }
c3_non_events = { k:[] for k in c3_intervals.columns }

sformat = "%Y-%m-%d %H:%M:%S"
    
last_c2_end = None
last_c3_end = None

count = 0
for event in cme_data.iterrows():

    # grab date-time-duration info for our event
    cme_start_date = [int(t) for t in event[1]['Date'].split('/')]
    cme_start_time = [int(t) for t in event[1]['Time'].split(':')]
    cme_c3_delta = timedelta(seconds=event[1]['StartTime_C3_Delta'])
    cme_c2_duration = timedelta(seconds=float(event[1]['Duration_C2']))
    cme_c3_duration = timedelta(seconds=float(event[1]['Duration_C3']))
    
    # convert to Python datetime for easier slicing
    cme_date = date (year=cme_start_date[0], month=cme_start_date[1], day=cme_start_date[2])
    cme_time = time(hour=cme_start_time[0],minute=cme_start_time[1],second=cme_start_time[2])

    c2_start = datetime.combine(cme_date,cme_time)
    c2_end   = c2_start + cme_c2_duration

    c3_start = c2_start + cme_c3_delta
    c3_end   = c3_start + cme_c3_duration
    
    # we have free interval for C2, C3 based on prior event end and 
    # the start of the new event
    if last_c2_end != None:
        for c2_ne in c2_intervals.loc[last_c2_end.strftime(sformat):c2_start.strftime(sformat)].iterrows():
            for col in c2_intervals.columns:
                c2_non_events[col].append(c2_ne[1][col])
                
    if last_c3_end != None:
        for c3_ne in c3_intervals.loc[last_c3_end.strftime(sformat):c3_start.strftime(sformat)].iterrows():
            for col in c3_intervals.columns:
                c3_non_events[col].append(c3_ne[1][col])

    last_c2_end = c2_end
    last_c3_end = c3_end


In [15]:
foo = c2_intervals[c2_intervals['datetime']>'2000-07-11']
bar = foo[foo['datetime']<'2000-07-13']
bar

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime,telescope
datetime,,,,,,,,,
2000-07-11 00:06:06,25072264.fts,2000-07-11 00:06:06,2000/07/11,1024,1024,Orang,Clear,25.1,C2
2000-07-11 00:25:16,25072265.fts,2000-07-11 00:25:16,2000/07/11,1024,1024,Orang,Clear,26.6,C2
2000-07-11 00:49:20,25072266.fts,2000-07-11 00:49:20,2000/07/11,1024,1024,Orang,Clear,25.1,C2
2000-07-11 01:26:38,25072267.fts,2000-07-11 01:26:38,2000/07/11,1024,1024,Orang,Clear,25.1,C2
2000-07-11 01:49:25,25072268.fts,2000-07-11 01:49:25,2000/07/11,1024,1024,Orang,Clear,25.0,C2
...,...,...,...,...,...,...,...,...,...
2000-07-12 22:05:40,25072402.fts,2000-07-12 22:05:40,2000/07/12,1024,1024,Orang,Clear,25.2,C2
2000-07-12 22:29:17,25072403.fts,2000-07-12 22:29:17,2000/07/12,1024,1024,Orang,Clear,25.0,C2
2000-07-12 23:05:25,25072404.fts,2000-07-12 23:05:25,2000/07/12,1024,1024,Orang,Clear,25.1,C2


In [16]:
# c2_intervals.loc['1996-01-31 00:00:00':'2000-02-12 00:00:00']

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime,telescope
datetime,,,,,,,,,
1996-01-31 01:55:44,25000288.fts,1996-01-31 01:55:44,1996/01/31,1024,1024,Orang,Clear,25.1,C2
1996-01-31 02:20:38,25000289.fts,1996-01-31 02:20:38,1996/01/31,1024,1024,Orang,Clear,25.0,C2
1996-01-31 04:32:28,25000291.fts,1996-01-31 04:32:28,1996/01/31,1024,1024,Orang,Clear,25.0,C2
1996-01-31 05:42:43,25000292.fts,1996-01-31 05:42:43,1996/01/31,1024,1024,Orang,Clear,25.1,C2
1996-01-31 06:52:13,25000293.fts,1996-01-31 06:52:13,1996/01/31,1024,1024,Orang,Clear,25.2,C2
...,...,...,...,...,...,...,...,...,...
2000-02-11 22:05:52,25061695.fts,2000-02-11 22:05:52,2000/02/11,1024,1024,Orang,Clear,25.1,C2
2000-02-11 22:29:47,25061696.fts,2000-02-11 22:29:47,2000/02/11,1024,1024,Orang,Clear,25.3,C2
2000-02-11 23:06:00,25061697.fts,2000-02-11 23:06:00,2000/02/11,1024,1024,Orang,Clear,25.1,C2


In [17]:
c2_non_events = pd.DataFrame(c2_non_events)
c2_non_events

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime,telescope
0,25000300.fts,1996-01-31 18:48:55,1996/01/31,512,512,Orang,Clear,6.4,C2
1,25000301.fts,1996-02-01 00:43:07,1996/02/01,1024,1024,Orang,Clear,25.0,C2
2,25000302.fts,1996-02-01 01:07:04,1996/02/01,1024,1024,Orang,Clear,25.1,C2
3,25000303.fts,1996-02-01 02:21:56,1996/02/01,1024,1024,Orang,Clear,25.4,C2
4,25000304.fts,1996-02-01 03:36:53,1996/02/01,1024,1024,Orang,Clear,25.1,C2
...,...,...,...,...,...,...,...,...,...
432488,25649100.fts,2017-08-31 22:34:39,2017/08/31,1024,1024,Orang,Clear,25.1,C2
432489,25649101.fts,2017-08-31 23:10:42,2017/08/31,1024,1024,Orang,Clear,25.1,C2
432490,25649102.fts,2017-08-31 23:22:37,2017/08/31,1024,1024,Orang,Clear,25.5,C2
432491,25649103.fts,2017-08-31 23:34:50,2017/08/31,1024,1024,Orang,Clear,25.1,C2


In [18]:
c3_non_events = pd.DataFrame(c3_non_events)
c3_non_events

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime,telescope
0,35003726.fts,1996-02-02 02:26:29,1996/02/02,1024,1024,Clear,Clear,19.1,C3
1,35003727.fts,1996-02-02 03:02:49,1996/02/02,1024,1024,Clear,Clear,10.3,C3
2,35003728.fts,1996-02-02 04:10:18,1996/02/02,1024,1024,Clear,Clear,19.1,C3
3,35003729.fts,1996-02-02 05:17:45,1996/02/02,1024,1024,Clear,Clear,19.9,C3
4,35003730.fts,1996-02-02 06:24:17,1996/02/02,1024,1024,Clear,Clear,19.0,C3
...,...,...,...,...,...,...,...,...,...
145079,35513408.fts,2017-08-30 09:53:40,2017/08/30,1024,1024,Clear,Clear,17.6,C3
145080,35513409.fts,2017-08-30 10:05:44,2017/08/30,1024,1024,Clear,Clear,17.6,C3
145081,35513410.fts,2017-08-30 10:17:41,2017/08/30,1024,1024,Clear,Clear,17.6,C3
145082,35513411.fts,2017-08-30 10:29:51,2017/08/30,1024,1024,Clear,Clear,17.6,C3


In [19]:
# save it 
c2_non_events.to_csv(C2_non_CME_Catalog_Filename, index=False)
c3_non_events.to_csv(C3_non_CME_Catalog_Filename, index=False)

In [20]:
event_catalog  = result
test_time = event_catalog['datetime'][1]
test_time

Timestamp('1996-01-31 09:11:14')

In [21]:
len(event_catalog), len(c2_intervals)

(593209, 554838)

In [22]:
# build out a filtered catlog so that we match only stuff we can accept
# for differencing later on (e.g. same params as before)
c2_a = c2_intervals[c2_intervals['expTime']>24.]
c2_b = c2_a[c2_a['expTime']<27.]
c2_c = c2_b[c2_b['Filter'].str.match('Orang')]
c2_d = c2_c[c2_c['NumCols'] == 1024]
c2_filt_cat = c2_d[c2_d['NumRows'] == 1024]

c2_filt_cat

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime,telescope
datetime,,,,,,,,,
1996-01-28 17:18:10,25000254.fts,1996-01-28 17:18:10,1996/01/28,1024,1024,Orang,Clear,25.0,C2
1996-01-30 03:54:18,25000278.fts,1996-01-30 03:54:18,1996/01/30,1024,1024,Orang,Clear,25.2,C2
1996-01-30 04:38:27,25000279.fts,1996-01-30 04:38:27,1996/01/30,1024,1024,Orang,Clear,25.0,C2
1996-01-30 05:56:07,25000280.fts,1996-01-30 05:56:07,1996/01/30,1024,1024,Orang,Clear,25.1,C2
1996-01-30 07:13:53,25000281.fts,1996-01-30 07:13:53,1996/01/30,1024,1024,Orang,Clear,25.1,C2
...,...,...,...,...,...,...,...,...,...
2017-08-31 22:34:39,25649100.fts,2017-08-31 22:34:39,2017/08/31,1024,1024,Orang,Clear,25.1,C2
2017-08-31 23:10:42,25649101.fts,2017-08-31 23:10:42,2017/08/31,1024,1024,Orang,Clear,25.1,C2
2017-08-31 23:22:37,25649102.fts,2017-08-31 23:22:37,2017/08/31,1024,1024,Orang,Clear,25.5,C2


In [23]:
d = c2_filt_cat[c2_filt_cat['datetime']<test_time]
foo = d.iloc[0]
type(foo), foo

(pandas.core.series.Series,
 filename            25000254.fts
 datetime     1996-01-28 17:18:10
 date-obs              1996/01/28
 NumCols                     1024
 NumRows                     1024
 Filter                     Orang
 Polarizer                  Clear
 expTime                     25.0
 telescope                     C2
 Name: 1996-01-28 17:18:10, dtype: object)